### Predicting sentiment from product reviews
The goal of this assignment is to explore logistic regression and feature engineering with existing GraphLab Create functions.

In this assignment, you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative. You will:

Use SFrames to do some feature engineering Train a logistic regression model to predict the sentiment of product reviews. Inspect the weights (coefficients) of a trained logistic regression model. Make a prediction (both class and probability) of sentiment for a new product review. Given the logistic regression weights, predictors and ground truth labels, write a function to compute the accuracy of the model. Inspect the coefficients of the logistic regression model and interpret their meanings. Compare multiple logistic regression models.

In [1]:
import pandas as pd
import numpy as np

In [2]:
products = pd.read_csv('amazon_baby.csv')

In [3]:
products.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


### Perform text cleaning
We start by removing punctuation, so that words "cake." and "cake!" are counted as the same word.

Write a function remove_punctuation that strips punctuation from a line of text Apply this function to every element in the review column of products, and save the result to a new column review_clean. Refer to your tool's manual for string processing capabilities. Python lets us express the operation in a succinct way, as follows:

In [4]:
import string

In [5]:
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

In [6]:
products = products.fillna({'review':''})  # fill in N/A's in the review column

In [7]:
products['review_clean'] = products['review'].apply(lambda x: remove_punctuation(x))

### Extract Sentiments
We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment.

In [8]:
products = products[products['rating'] != 3].copy(deep=True)

Now, we will assign reviews with a rating of 4 or higher to be positive reviews, while the ones with rating of 2 or lower are negative. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label. A good way is to create an anonymous function that converts a rating into a class label and then apply that function to every element in the rating column.

In [9]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)

In [10]:
products.head()

,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,1


In [11]:
import json

def readjson(path):
    try:
        with open(path,'r') as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        raise FileNotFoundError( "Please check the path and try agaian!!!")


In [12]:
train_list = readjson(path='module-2-assignment-train-idx.json')
test_list = readjson(path='module-2-assignment-test-idx.json')

In [13]:
train_data = products.iloc[train_list].copy(deep=True)
test_data = products.iloc[test_list].copy(deep=True)

### Build the word count vector for each review
We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as bag-of-word features. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction. Compute the occurrences of the words in each review and collect them into a row vector. Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix train_matrix. Using the same mapping between words and columns, convert the test data into a sparse matrix test_matrix.

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
# Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

### Train a sentiment classifier with logistic regression
We will now use logistic regression to create a sentiment classifier on the training data.

Learn a logistic regression classifier using the training data. If you are using scikit-learn, you should create an instance of the LogisticRegression class and then call the method fit() to train the classifier. This model should use the sparse word count matrix (train_matrix) as features and the column sentiment of train_data as the target. Use the default values for other parameters. Call this model sentiment_model.

In [15]:
from sklearn import datasets, linear_model
from sklearn.linear_model import LogisticRegression

In [16]:
sentiment_model = LogisticRegression()
sentiment_model.fit(train_matrix, train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

**Quiz question: How many weights are >= 0?**

In [17]:
print (np.sum(sum(sentiment_model.coef_ >= 0)))

85759


### Making predictions with logistic regression
Now that a model is trained, we can make predictions on the test data. In this section, we will explore this in the context of 3 data points in the test data. Take the 11th, 12th, and 13th data points in the test data and save them to sample_test_data. The following cell extracts the three data points from the SFrame test_data and print their content:

In [18]:
sample_test_data = test_data[10:13].copy(deep=True)
for i in sample_test_data.index:
    print (sample_test_data.loc[i]['review_clean'], end = '\n\n')


Absolutely love it and all of the Scripture in it  I purchased the Baby Boy version for my grandson when he was born and my daughterinlaw was thrilled to receive the same book again

Would not purchase again or recommend The decals were thick almost plastic like and were coming off the wall as I was applying them The would NOT stick Literally stayed stuck for about 5 minutes then started peeling off

Was so excited to get this product for my baby girls bedroom  When I got it the back is NOT STICKY at all  Every time I walked into the bedroom I was picking up pieces off of the floor  Very very frustrating  Ended up having to super glue it to the wallvery disappointing  I wouldnt waste the time or money on it



In [19]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
print (scores)

[  5.59930746  -3.14046638 -10.40387188]


### Prediciting Sentiment
These scores can be used to make class predictions as follows:

Using scores, write code to calculate predicted labels for sample_test_data.

Checkpoint: Make sure your class predictions match with the ones obtained from sentiment_model. The logistic regression classifier in scikit-learn comes with the predict function for this purpose.

In [20]:
sentiment_model.predict(sample_test_matrix)

array([ 1, -1, -1])

### Probability Predictions
Recall from the lectures that we can also calculate the probability predictions from the scores using:

Using the scores calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range [0, 1].

Checkpoint: Make sure your probability predictions match the ones obtained from sentiment_model.

In [21]:
probability = 1/(1+np.exp(-scores))

In [22]:
probability

array([9.96313217e-01, 4.14685772e-02, 3.03139609e-05])

### Find the most positive (and negative) review
We now turn to examining the full test dataset, test_data, and use sklearn.linear_model.LogisticRegression to form predictions on all of the test data points.

Using the sentiment_model, find the 20 reviews in the entire test_data with the highest probability of being classified as a positive review. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:

Make probability predictions on test_data using the sentiment_model. Sort the data according to those predictions and pick the top 20.

In [23]:
test_matrix = vectorizer.transform(test_data['review_clean'])
scores_test = sentiment_model.decision_function(test_matrix)
predictions = 1/(1+np.exp(-scores_test))

In [24]:
test_data.columns

Index(['name', 'review', 'rating', 'review_clean', 'sentiment'], dtype='object')

In [25]:
test_data['predictions'] = predictions

In [26]:
test_data.sort_values('predictions', ascending = False).head(20)

,name,review,rating,review_clean,sentiment,predictions
180646,Mamas &amp; Papas 2014 Urbo2 Stroller - Black,After much research I purchased an Urbo2. It's...,4,After much research I purchased an Urbo2 Its e...,1,1.0
87017,Baby Einstein Around The World Discovery Center,I am so HAPPY I brought this item for my 7 mon...,5,I am so HAPPY I brought this item for my 7 mon...,1,1.0
147949,"Baby Jogger City Mini GT Single Stroller, Shad...","Amazing, Love, Love, Love it !!! All 5 STARS a...",5,Amazing Love Love Love it All 5 STARS all the...,1,1.0
97325,Freemie Hands-Free Concealable Breast Pump Col...,I absolutely love this product. I work as a C...,5,I absolutely love this product I work as a Cu...,1,1.0
168081,Buttons Cloth Diaper Cover - One Size - 8 Colo...,"We are big Best Bottoms fans here, but I wante...",4,We are big Best Bottoms fans here but I wanted...,1,1.0
119182,Roan Rocco Classic Pram Stroller 2-in-1 with B...,Great Pram Rocco!!!!!!I bought this pram from ...,5,Great Pram RoccoI bought this pram from Europe...,1,1.0
52631,Evenflo X Sport Plus Convenience Stroller - Ch...,After seeing this in Parent's Magazine and rea...,5,After seeing this in Parents Magazine and read...,1,1.0
66059,"Evenflo 6 Pack Classic Glass Bottle, 4-Ounce",It's always fun to write a review on those pro...,5,Its always fun to write a review on those prod...,1,1.0
114796,"Fisher-Price Cradle 'N Swing, My Little Snuga...",My husband and I cannot state enough how much ...,5,My husband and I cannot state enough how much ...,1,1.0
168697,Graco FastAction Fold Jogger Click Connect Str...,Graco's FastAction Jogging Stroller definitely...,5,Gracos FastAction Jogging Stroller definitely ...,1,1.0


In [27]:
test_data.sort_values('predictions', ascending = True).head(20)

,name,review,rating,review_clean,sentiment,predictions
16042,Fisher-Price Ocean Wonders Aquarium Bouncer,We have not had ANY luck with Fisher-Price pro...,2,We have not had ANY luck with FisherPrice prod...,-1,8.781182e-16
120209,Levana Safe N'See Digital Video Baby Monitor w...,This is the first review I have ever written o...,1,This is the first review I have ever written o...,-1,1.899745e-15
77072,Safety 1st Exchangeable Tip 3 in 1 Thermometer,I thought it sounded great to have different t...,1,I thought it sounded great to have different t...,-1,7.962844e-14
48694,Adiri BPA Free Natural Nurser Ultimate Bottle ...,I will try to write an objective review of the...,2,I will try to write an objective review of the...,-1,1.391760e-13
155287,VTech Communications Safe &amp; Sounds Full Co...,"This is my second video monitoring system, the...",1,This is my second video monitoring system the ...,-1,1.637989e-13
94560,The First Years True Choice P400 Premium Digit...,Note: we never installed batteries in these un...,1,Note we never installed batteries in these uni...,-1,4.619406e-13
53207,Safety 1st High-Def Digital Monitor,We bought this baby monitor to replace a diffe...,1,We bought this baby monitor to replace a diffe...,-1,3.366419e-11
81332,Cloth Diaper Sprayer--styles may vary,I bought this sprayer out of desperation durin...,1,I bought this sprayer out of desperation durin...,-1,4.175884e-11
113995,Motorola Digital Video Baby Monitor with Room ...,DO NOT BUY THIS BABY MONITOR!I purchased this ...,1,DO NOT BUY THIS BABY MONITORI purchased this m...,-1,9.690073e-11
10677,Philips AVENT Newborn Starter Set,"It's 3am in the morning and needless to say, t...",1,Its 3am in the morning and needless to say thi...,-1,1.120701e-10


### Compute accuracy of the classifier
We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by

This can be computed as follows:

Step 1: Use the sentiment_model to compute class predictions.

Step 2: Count the number of data points when the predicted class labels match the ground truth labels.

Step 3: Divide the total number of correct predictions by the total number of data points in the dataset.

In [28]:
predicted_sentiment = sentiment_model.predict(test_matrix)

In [29]:
test_data['predicted_sentiment'] = predicted_sentiment

In [30]:
test_data['diff_sentiment'] = predicted_sentiment - test_data['sentiment']

In [31]:
acc = np.sum(sum(test_data['diff_sentiment'] == 0))
print (acc)

31082


In [32]:
total = len(test_data.index)
print (total)

33336


In [33]:
print (round(float(acc)/float(total),2))

0.93


### Learn another classifier with fewer words
There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subet of words that occur in the reviews. For this assignment, we selected 20 words to work with. These are:

In [34]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

Compute a new set of word count vectors using only these words. The CountVectorizer class has a parameter that lets you limit the choice of words when building word count vectors:

In [35]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])

### Train a logistic regression model on a subset of data
Now build a logistic regression classifier with train_matrix_word_subset as features and sentiment as the target.

Call this model simple_model.

In [36]:
simple_model = LogisticRegression()

In [37]:
simple_model.fit(train_matrix_word_subset, train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Let us inspect the weights (coefficients) of the simple_model. First, build a table to store (word, coefficient) pairs. If you are using SFrame with scikit-learn, you can combine words with coefficients by running

In [38]:
simple_model.coef_.flatten()

array([ 1.36368976,  0.94399959,  1.19253827,  0.08551278,  0.52018576,
        1.50981248,  1.67307389,  0.50376046,  0.19090857,  0.05885467,
       -1.65157634, -0.20956286, -0.51137963, -2.03369861, -2.34829822,
       -0.62116877, -0.32055624, -0.89803074, -0.36216674, -2.10933109])

In [39]:
simple_model_coef_table = pd.DataFrame({'word':significant_words,
                                         'coefficient':simple_model.coef_.flatten()})

In [40]:
simple_model_coef_table.sort_values('coefficient',ascending = False)

,word,coefficient
6,loves,1.673074
5,perfect,1.509812
0,love,1.363690
2,easy,1.192538
1,great,0.944000
4,little,0.520186
7,well,0.503760
8,able,0.190909
3,old,0.085513
9,car,0.058855


In [41]:
sum(simple_model_coef_table['coefficient']>=0)

10

In [42]:
vocab = list(vectorizer.vocabulary_.keys())

In [43]:
vocab

['it',
 'came',
 'early',
 'and',
 'was',
 'not',
 'disappointed',
 'i',
 'love',
 'planet',
 'wise',
 'bags',
 'now',
 'my',
 'wipe',
 'holder',
 'keps',
 'osocozy',
 'wipes',
 'moist',
 'does',
 'leak',
 'highly',
 'recommend',
 'very',
 'soft',
 'comfortable',
 'warmer',
 'than',
 'looksfit',
 'the',
 'full',
 'size',
 'bed',
 'perfectlywould',
 'to',
 'anyone',
 'looking',
 'for',
 'this',
 'type',
 'of',
 'quilt',
 'is',
 'a',
 'product',
 'well',
 'worth',
 'purchase',
 'have',
 'found',
 'anything',
 'else',
 'like',
 'positive',
 'ingenious',
 'approach',
 'losing',
 'binky',
 'what',
 'most',
 'about',
 'how',
 'much',
 'ownership',
 'daughter',
 'has',
 'in',
 'getting',
 'rid',
 'she',
 'so',
 'proud',
 'herself',
 'loves',
 'her',
 'little',
 'fairy',
 'artwork',
 'chart',
 'back',
 'clever',
 'tool',
 'all',
 'kids',
 'cried',
 'nonstop',
 'when',
 'tried',
 'ween',
 'them',
 'off',
 'their',
 'pacifier',
 'until',
 'thumbuddy',
 'puppet',
 'an',
 'easy',
 'way',
 'work',


In [44]:
coeffs = {vocab[i]: c for i, c in enumerate(sentiment_model.coef_[0])}
new_dic = {k:v for k, v in coeffs.items() if k in significant_words}
new_table = pd.DataFrame(new_dic.items(), columns=['word', 'new_coefficient'])
new_table_coeff = pd.merge(simple_model_coef_table, new_table, how = 'left', on = 'word' )
new_table_coeff = new_table_coeff[new_table_coeff['coefficient']>=0].copy(deep=True)
print(sum(new_table_coeff['new_coefficient'] < 0))
new_table_coeff


3


,word,coefficient,new_coefficient
0,love,1.363690,0.251931
1,great,0.944000,0.068691
2,easy,1.192538,-0.004988
3,old,0.085513,0.008205
4,little,0.520186,-0.294920
5,perfect,1.509812,-0.616341
6,loves,1.673074,0.008729
7,well,0.503760,0.000018
8,able,0.190909,0.207305
9,car,0.058855,0.050010


### Comparing models
We will now compare the accuracy of the sentiment_model and the simple_model.

First, compute the classification accuracy of the sentiment_model on the train_data.

Now, compute the classification accuracy of the simple_model on the train_data.

the accuracy of the sentiment_model and the simple mode

In [45]:
predicted_sentiment_train_sentiment = sentiment_model.predict(train_matrix)
train_data['predicted_sentiment_ts'] = predicted_sentiment_train_sentiment
acc_ts = round(float(sum(train_data['predicted_sentiment_ts'] == train_data['sentiment']))/len(train_data.index),2)

In [46]:
predicted_simple_train_sentiment = simple_model.predict(train_matrix_word_subset)
train_data['predicted_simple_ts'] = predicted_simple_train_sentiment
acc_tsimple = round(float(sum(train_data['predicted_simple_ts'] == train_data['sentiment']))/len(train_data.index),2)

In [47]:
acc_ts > acc_tsimple

True

In [48]:
acc_tsimple

0.87

In [49]:
acc_ts

0.97

compare the accuracy of the sentiment_model and the simple model

In [50]:
predicted_sentiment_test_sentiment = sentiment_model.predict(test_matrix)
test_data['predicted_sentiment_ts'] = predicted_sentiment_test_sentiment
acc_ts_test = round(float(sum(test_data['predicted_sentiment_ts'] == test_data['sentiment']))/len(test_data.index),2)
acc_ts_test

0.93

In [51]:
predicted_simple_test_sentiment = simple_model.predict(test_matrix_word_subset)
test_data['predicted_simple_ts'] = predicted_simple_test_sentiment
acc_tsimple_test = round(float(sum(test_data['predicted_simple_ts'] == test_data['sentiment']))/len(test_data.index),2)
acc_tsimple_test

0.87

In [52]:
acc_ts_test > acc_tsimple_test

True

### The majority class model

In [53]:
print ("Total Positive Sentiments: {}".format(sum(train_data['sentiment'] == 1)))
print ("Total Negative Sentiments: {}".format(sum(train_data['sentiment'] == -1)))


Total Positive Sentiments: 112164
Total Negative Sentiments: 21252


In [54]:
round(float(sum(test_data['sentiment'] ==1))/len(test_data.index),2)

0.84